In [1]:
!pip install ultralytics > /dev/null

In [2]:
import os
import shutil
import random
from ultralytics import YOLO
from sklearn.model_selection import train_test_split

dataset_path = "/kaggle/input/data30/dataset"
output_path = "/kaggle/working"

os.makedirs(os.path.join(output_path, "train/images"), exist_ok=True)
os.makedirs(os.path.join(output_path, "train/labels"), exist_ok=True)
os.makedirs(os.path.join(output_path, "val/images"), exist_ok=True)
os.makedirs(os.path.join(output_path, "val/labels"), exist_ok=True)

image_files = [f for f in os.listdir(os.path.join(dataset_path, "images")) 
              if f.endswith(('.jpg', '.jpeg', '.png'))]

train_files, val_files = train_test_split(image_files, test_size=0.2, random_state=40)

def copy_files(files, split_type):
    for file in files:
        img_src = os.path.join(dataset_path, "images", file)
        img_dst = os.path.join(output_path, split_type, "images", file)
        shutil.copy(img_src, img_dst)

        label_file = os.path.splitext(file)[0] + '.txt'
        label_src = os.path.join(dataset_path, "labels", label_file)
        label_dst = os.path.join(output_path, split_type, "labels", label_file)
        if os.path.exists(label_src):
            shutil.copy(label_src, label_dst)

copy_files(train_files, "train")
copy_files(val_files, "val")

data_yaml = f"""
path: {output_path}
train: train/images
val: val/images

nc: 1
names: ['human']
"""

with open('data.yaml', 'w') as file:
    file.write(data_yaml)



Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
model = YOLO("yolo12n.pt")

model.train(
    data='data.yaml',       # Путь к конфигурационному файлу данных 
                           # Содержит пути к обучающим/валидационным данным, имена классов
                           # Обязателен для определения структуры задачи детекции
    
    epochs=1,               # Количество полных проходов через весь датасет
                           # Для отладки используют 1 эпоху, реальное обучение требует 50-500+
                           # Недостаточное количество эпох → недообучение, избыточное → переобучение
    
    batch=4,                # Количество изображений в одной итерации (batch size)
                           # Малый batch (4-16): меньше потребление памяти, но шумные градиенты
                           # Крупный batch (>32): стабильнее сходимость, но требует больше VRAM
    
    imgsz=800,              # Размер входных изображений (пиксели)
                           # Большие размеры (640-1280) улучшают детекцию мелких объектов
                           # Увеличивает точность за счет роста вычислительной нагрузки
    
    patience=15,            # Количество эпох без улучшений до ранней остановки
                           # Прерывает обучение при стагнации метрик (экономия ресурсов)
                           # Оптимально: 10-25% от общего числа эпох
    
    save_period=1,          # Сохранять чекпоинт каждую эпоху
                           # Позволяет анализировать промежуточные результаты
                           # Для длительного обучения можно увеличить (напр., до 5-10)
    
    workers=4,              # Количество подпроцессов для загрузки данных
                           # Рекомендуется: 4-8 (зависит от CPU). Слишком много → конфликты
                           # Ускоряет подготовку батчей, предотвращает "бутылочное горло" на CPU
    #Параметры оптимизатора
    optimizer='SGD',        # Алгоритм оптимизации (Stochastic Gradient Descent)
                           # Альтернативы: 'Adam', 'AdamW', 'RMSprop'
                           # SGD лучше для задач детекции, но требует тонкой настройки
    
    momentum=0.937,         # Инерция градиента (0.9-0.95)
                           # Ускоряет сходимость в минимуме, снижает колебания
                           # Слишком высокое значение → "проскакивание" оптимума
    
    lr0=0.001,              # Стартовый learning rate 
                           # Критичный параметр! Типичные значения: 0.01 - 0.0001
                           # Слишком высокий → дивергенция, слишком низкий → медленная сходимость
    
    weight_decay=0.0005,    # Коэффициент L2-регуляризации (0.0001-0.005)
                           # "Штрафует" большие веса → предотвращает переобучение
                           # Баланс между регуляризацией и сохранением емкости модели
    
    cos_lr=True,            # Использовать косинусный ансамбль learning rate
                           # Плавно уменьшает LR по косинусоиде (лучше чем step decay)
                           # Повышает стабильность финальной стадии обучения
    
    #Аугментации изображений
    ## Цветовые искажения
    hsv_h=0.02,             # Изменение оттенка (диапазон: 0.0-0.1)
                           # Имитирует разную освещенность/цветопередачу камер
    hsv_s=0.8,              # Коррекция насыщенности (0.0-1.0)
                           # Помогает с разными условиями освещения
    hsv_v=0.5,              # Коррекция яркости (0.0-1.0)
                           # Устойчивость к теням/пересветам
    
    ## Геометрические трансформации
    flipud=0.4,             # Вертикальное отражение (вероятность 40%)
                           # Полезно для объектов с инвариантной ориентацией (лица, автомобили)
    fliplr=0.6,             # Горизонтальное отражение (60%)
                           # Стандартная практика для большинства объектов
    translate=0.15,         # Случайный сдвиг (±15% от размера изображения)
                           # Имитирует неточное кадрирование/сдвиг камеры
    scale=0.4,              # Масштабирование (±40%)
                           # Учит распознавать объекты разных размеров
    shear=0.02,             # Наклон (±2 градуса)
                           # Компенсирует перспективные искажения
    perspective=0.001,      # Перспективные искажения (0.001-0.0001)
                           # Эмулирует 3D-ракурсы
    
    ## Составные аугментации
    mosaic=1.0,             # Вероятность применения мозаики (100%)
                           # Склейка 4 изображений → улучшает детекцию контекста
    mixup=0.2,              # Вероятность смешивания изображений (20%)
                           # Наложение двух изображений → повышает робастность
)

100%|██████████| 5.34M/5.34M [00:00<00:00, 118MB/s]


Ultralytics 8.3.170 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1, erasing=0.4, exist_ok=False, fliplr=0.6, flipud=0.4, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.02, hsv_s=0.8, hsv_v=0.5, imgsz=800, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.2, mode=train, model=yolo12n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=SGD, overlap_mask=True, patience=15, perspective=0.001, plots=True, pose=12.0, pretrained=True, pro

100%|██████████| 755k/755k [00:00<00:00, 27.8MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  2    180864  ultralytics.nn.modules.block.A2C2f           [128, 128, 2, True, 4]        
  7                  -1  1    295424  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 104MB/s]


...

        1/1      1.71G      1.997      3.771      1.654         10        800: 100%|██████████| 862/862 [02:44<00:00,  5.24it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 108/108 [00:11<00:00,  9.37it/s]
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        863       1156      0.522       0.47      0.436      0.206

1 epochs completed in 0.050 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 5.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 5.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.170 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLOv12n summary (fused): 159 layers, 2,556,923 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 108/108 [00:09<00:00, 11.86it/s]
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        863       1156      0.521       0.47      0.437      0.206
Speed: 0.3ms preprocess, 5.4ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ec1fc2dc590>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 